# Explorations des systemes de recommendations

In [3]:
import pandas as pd

DATA_PATH = r"..\data\news-portal-user-interactions-by-globocom\clicks_sample.csv"

df = pd.read_csv(DATA_PATH, sep=',')
df.head()

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2


# Recomendation basé sur la popularité

In [31]:
def get_popular_recommendations(dataframe, top_n=5):
    """
    Recommende les articles les plus populaires

    Args:
        dataframe: pandas dataframe des articles et des clicks
        top_n: Nombre d'articles à recommander (par défaut 5).

    Returns:
        list: Une liste de top_n click_article_id les plus populaires.
    """
    # Compter le nombre de clics par article
    popular_articles = dataframe['click_article_id'].value_counts()

    # Sélectionner les top_n articles les plus populaires
    top_articles = popular_articles.head(top_n).index.tolist()

    return top_articles


In [32]:
get_popular_recommendations(clicks_df)

[119592, 96663, 108854, 284847, 235840]

# Filtrage Collaboratif Basé sur les Items (Item-Based Collaborative Filtering)


In [36]:
import pandas as pd
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def load_and_prepare_data(clicks_file_path):
    """
    Charge les données de clics et les prépare pour le filtrage collaboratif.

    Args:
        clicks_file_path (str): Chemin vers le fichier CSV des clics.

    Returns:
        pd.DataFrame: Le DataFrame des clics nettoyé.
        pd.DataFrame: La matrice d'interaction utilisateur-article binaire (users x articles).
    """
    # Charger les données
    clicks_df = pd.read_csv(clicks_file_path)

    # Convertir le timestamp en datetime pour les analyses futures si nécessaire
    clicks_df['click_timestamp'] = pd.to_datetime(clicks_df['click_timestamp'])

    # Nettoyage basique (suppression des lignes avec article_id manquant)
    clicks_df = clicks_df.dropna(subset=['click_article_id'])

    # Construire la matrice d'interaction utilisateur-article
    # Pivot pour avoir des lignes=user_id, colonnes=click_article_id
    # On compte le nombre de clics (ou on peut juste utiliser 1 pour binaire)
    interaction_matrix = clicks_df.pivot_table(
        index='user_id',
        columns='click_article_id',
        values='click_timestamp', # On peut utiliser n'importe quelle colonne pour compter
        aggfunc='count', # Compter les clics
        fill_value=0    # Remplir les absences de clic avec 0
    )

    # Convertir en binaire (a cliqué / n'a pas cliqué)
    interaction_matrix_binary = interaction_matrix.astype(bool).astype(int)

    return clicks_df, interaction_matrix_binary

def get_item_based_collaborative_recommendations(user_id, interaction_matrix, top_n=5):
    """
    Recommende des articles basés sur les articles consultés par l'utilisateur
    en utilisant le filtrage collaboratif basé sur les items.

    Args:
        user_id: L'ID de l'utilisateur pour lequel on veut recommander.
        interaction_matrix (pd.DataFrame): La matrice d'interaction utilisateur-article binaire.
        top_n (int): Nombre d'articles à recommander (par défaut 5).

    Returns:
        list: Une liste de top_n click_article_id recommandés.
    """
    # Vérifier si l'utilisateur existe dans la matrice
    if user_id not in interaction_matrix.index:
        print(f"User ID {user_id} non trouvé dans les données d'entraînement. Impossible de recommander.")
        return []

    # Récupérer les articles que l'utilisateur a consultés (valeur > 0 dans sa ligne)
    user_interactions = interaction_matrix.loc[user_id]
    user_clicked_articles = user_interactions[user_interactions > 0].index.tolist()

    if not user_clicked_articles:
        print(f"L'utilisateur {user_id} n'a consulté aucun article dans les données d'entraînement.")
        return []

    print(f"Articles consultés par l'utilisateur {user_id}: {user_clicked_articles}")

    # Calculer la matrice de similarité entre les ARTICLES
    # On transpose la matrice pour que les articles soient les "lignes"
    articles_matrix = interaction_matrix.T # Shape: (n_articles, n_users)
    # Calculer la similarité cosinus entre les articles
    # Résultat: une matrice (n_articles, n_articles)
    similarity_matrix = cosine_similarity(articles_matrix)

    # Convertir en DataFrame pour un accès facile avec les noms d'articles (colonne/article_id)
    similarity_df = pd.DataFrame(
        similarity_matrix,
        index=interaction_matrix.columns, # Les article_ids
        columns=interaction_matrix.columns # Les article_ids
    )

    # Calculer un score de recommandation pour chaque article potentiellement non lu
    scores = defaultdict(float)

    for article_consulte in user_clicked_articles:
        # Récupérer les similarités de l'article consulté avec TOUS les autres articles
        similarities_to_article = similarity_df[article_consulte]

        # Ajouter la similarité à chaque article (sauf ceux déjà consultés)
        for other_article_id in interaction_matrix.columns:
            if other_article_id != article_consulte and other_article_id not in user_clicked_articles:
                # Ajouter la similarité comme score
                scores[other_article_id] += similarities_to_article[other_article_id]

    # Trier les articles par score décroissant
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

    # Sélectionner les top_n articles les mieux notés
    recommended_articles = [article_id for article_id, score in sorted_scores[:top_n]]

    return recommended_articles



In [38]:
# --- Exemple d'utilisation ---
if __name__ == "__main__":
    # Charger les données
    clicks_df, interaction_matrix = load_and_prepare_data(DATA_PATH)

    print(f"Matrice d'interaction chargée. Forme: {interaction_matrix.shape}")
    print(f"Exemples de user_ids: {interaction_matrix.index[:5].tolist()}")
    print(f"Exemples de article_ids: {interaction_matrix.columns[:5].tolist()}")

    # Sélectionner un utilisateur existant pour tester
    # On prend le premier user_id de la matrice pour s'assurer qu'il existe
    if not interaction_matrix.empty:
        test_user_id = interaction_matrix.index[2]
        print(f"\n--- Test pour l'utilisateur : {test_user_id} ---")
        recommendations = get_item_based_collaborative_recommendations(test_user_id, interaction_matrix)
        print(f"Articles recommandés pour l'utilisateur {test_user_id}: {recommendations}")
    else:
        print("La matrice d'interaction est vide.")


Matrice d'interaction chargée. Forme: (707, 323)
Exemples de user_ids: [0, 1, 2, 3, 4]
Exemples de article_ids: [2137, 2662, 4243, 4658, 7947]

--- Test pour l'utilisateur : 2 ---
Articles consultés par l'utilisateur 2: [30970, 119592]
Articles recommandés pour l'utilisateur 2: [114349, 141945, 4243, 31862, 235325]


# Filtrage Collaboratif Basé sur les Sessions (Session-Based Recommendation).

In [39]:
import pandas as pd
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def load_and_prepare_session_data(clicks_file_path):
    """
    Charge les données de clics et les prépare pour la recommandation basée sur les sessions.

    Args:
        clicks_file_path (str): Chemin vers le fichier CSV des clics.

    Returns:
        pd.DataFrame: Le DataFrame des clics nettoyé, trié par session et timestamp.
    """
    # Charger les données
    clicks_df = pd.read_csv(clicks_file_path)

    # Convertir le timestamp en datetime pour les analyses futures si nécessaire
    clicks_df['click_timestamp'] = pd.to_datetime(clicks_df['click_timestamp'])

    # Nettoyage basique (suppression des lignes avec article_id manquant)
    clicks_df = clicks_df.dropna(subset=['click_article_id'])

    # Trier par session_id et click_timestamp pour ordonner les clics dans une session
    clicks_df = clicks_df.sort_values(by=['session_id', 'click_timestamp'])

    return clicks_df

def get_session_based_recommendations(user_id, clicks_df, top_n=5):
    """
    Recommende des articles basés sur la dernière session de l'utilisateur
    en utilisant une approche basée sur les sessions (similitude des sessions).

    Args:
        user_id: L'ID de l'utilisateur pour lequel on veut recommander.
        clicks_df (pd.DataFrame): Le DataFrame des clics nettoyé et trié.
        top_n (int): Nombre d'articles à recommander (par défaut 5).

    Returns:
        list: Une liste de top_n click_article_id recommandés.
    """
    # 1. Récupérer les sessions de l'utilisateur
    user_sessions_df = clicks_df[clicks_df['user_id'] == user_id]

    if user_sessions_df.empty:
        print(f"Aucune session trouvée pour l'utilisateur {user_id}.")
        return []

    # 2. Identifier la dernière session de l'utilisateur (la plus récente)
    last_session_id = user_sessions_df['session_id'].iloc[-1] # Dernière ligne après tri
    last_session_articles = user_sessions_df[user_sessions_df['session_id'] == last_session_id]['click_article_id'].tolist()

    print(f"Articles de la dernière session ({last_session_id}) pour l'utilisateur {user_id}: {last_session_articles}")

    if not last_session_articles:
        print(f"La dernière session de l'utilisateur {user_id} ne contient aucun article.")
        return []

    # 3. Trouver d'autres sessions (potentiellement d'autres utilisateurs) qui contiennent des articles similaires
    # On crée un dictionnaire pour stocker les sessions et les articles qu'elles contiennent
    sessions_articles = clicks_df.groupby('session_id')['click_article_id'].apply(list).to_dict()

    # 4. Calculer une "similarité" entre la dernière session de l'utilisateur et toutes les autres sessions
    # Pour simplifier, on utilise une similarité de Jaccard (taille de l'intersection / taille de l'union)
    # On pondère potentiellement les articles trouvés dans des sessions très similaires
    scores = defaultdict(float)

    last_session_set = set(last_session_articles)
    for other_session_id, other_articles_list in sessions_articles.items():
        if other_session_id == last_session_id:
            continue # Ne pas comparer la session avec elle-même

        other_session_set = set(other_articles_list)
        intersection = last_session_set.intersection(other_session_set)
        union = last_session_set.union(other_session_set)

        if union: # Éviter la division par zéro
            jaccard_similarity = len(intersection) / len(union)
        else:
            jaccard_similarity = 0

        # Si la similarité est suffisante (par exemple > 0), on pondère les articles de l'autre session
        if jaccard_similarity > 0:
            for article in other_session_set:
                # Ne pas recommander un article déjà vu dans la session courante
                if article not in last_session_set:
                    # Le score est la somme pondérée par la similarité de la session
                    scores[article] += jaccard_similarity

    # 5. Trier les articles par score décroissant
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

    # 6. Sélectionner les top_n articles les mieux notés
    recommended_articles = [article_id for article_id, score in sorted_scores[:top_n]]

    return recommended_articles

# --- Exemple d'utilisation ---
if __name__ == "__main__":
    # Charger les données
    clicks_df = load_and_prepare_session_data(DATA_PATH)

    print(f"DataFrame des clics chargé. Forme: {clicks_df.shape}")
    print(f"Exemples de user_ids: {clicks_df['user_id'].unique()[:5]}")
    print(f"Exemples de session_ids: {clicks_df['session_id'].unique()[:5]}")

    # Sélectionner un utilisateur existant pour tester
    # On prend le premier user_id du DataFrame pour s'assurer qu'il existe
    if not clicks_df.empty:
        test_user_id = clicks_df['user_id'].iloc[0]
        print(f"\n--- Test pour l'utilisateur : {test_user_id} ---")
        recommendations = get_session_based_recommendations(test_user_id, clicks_df)
        print(f"Articles recommandés pour l'utilisateur {test_user_id}: {recommendations}")
    else:
        print("Le DataFrame des clics est vide.")


DataFrame des clics chargé. Forme: (1883, 12)
Exemples de user_ids: [0 1 2 3 4]
Exemples de session_ids: [1506825423271737 1506825426267738 1506825435299739 1506825442704740
 1506825528135741]

--- Test pour l'utilisateur : 0 ---
Articles de la dernière session (1506825423271737) pour l'utilisateur 0: [157541, 68866]
Articles recommandés pour l'utilisateur 0: [119592, 108854, 96663, 284847, 235840]


# Filtrage Collaboratif Basé sur SVD (Décomposition en Valeurs Singulières)

In [41]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def load_and_prepare_svd_data(clicks_file_path):
    """
    Charge les données de clics et les prépare pour le filtrage collaboratif basé sur SVD.

    Args:
        clicks_file_path (str): Chemin vers le fichier CSV des clics.

    Returns:
        pd.DataFrame: Le DataFrame des clics nettoyé.
        pd.DataFrame: La matrice d'interaction utilisateur-article pondérée (users x articles).
                      Ici, on pondère par le nombre de clics.
        list: La liste des user_ids (index de la matrice).
        list: La liste des article_ids (colonnes de la matrice).
    """
    # Charger les données
    clicks_df = pd.read_csv(clicks_file_path)

    # Convertir le timestamp en datetime pour les analyses futures si nécessaire
    clicks_df['click_timestamp'] = pd.to_datetime(clicks_df['click_timestamp'])

    # Nettoyage basique (suppression des lignes avec article_id manquant)
    clicks_df = clicks_df.dropna(subset=['click_article_id'])

    # Construire la matrice d'interaction utilisateur-article
    # Pivot pour avoir des lignes=user_id, colonnes=click_article_id
    # On compte le nombre de clics par utilisateur et par article
    interaction_matrix = clicks_df.pivot_table(
        index='user_id',
        columns='click_article_id',
        values='click_timestamp', # On peut utiliser n'importe quelle colonne pour compter
        aggfunc='count', # Compter les clics
        fill_value=0    # Remplir les absences de clic avec 0
    )

    # Le type de données est crucial pour SVD (float)
    interaction_matrix = interaction_matrix.astype(float)

    user_ids = interaction_matrix.index.tolist()
    article_ids = interaction_matrix.columns.tolist()

    return clicks_df, interaction_matrix, user_ids, article_ids

def train_svd_model(interaction_matrix, n_components=50):
    """
    Entraîne un modèle SVD sur la matrice d'interaction.

    Args:
        interaction_matrix (pd.DataFrame): La matrice d'interaction utilisateur-article.
        n_components (int): Nombre de composantes (facteurs latents) pour SVD.

    Returns:
        TruncatedSVD: Le modèle SVD entraîné.
        np.ndarray: La matrice des utilisateurs projetés dans l'espace latent.
        np.ndarray: La matrice des articles projetés dans l'espace latent.
    """
    print(f"Entraînement du modèle SVD avec {n_components} composantes...")
    # Initialiser TruncatedSVD
    svd = TruncatedSVD(n_components=n_components, random_state=42)

    # Ajuster le modèle et transformer la matrice d'interaction
    # La matrice U est la matrice des utilisateurs dans l'espace latent
    U = svd.fit_transform(interaction_matrix.values)
    # La matrice V est la matrice des articles dans l'espace latent (transposée)
    V_T = svd.components_
    V = V_T.T

    print(f"SVD terminé. Forme de U (utilisateurs latent): {U.shape}, Forme de V (articles latent): {V.shape}")
    return svd, U, V

def get_svd_recommendations(user_id, svd_model, U_latent, V_latent, user_ids, article_ids, top_n=5):
    """
    Recommende des articles basés sur le modèle SVD.

    Args:
        user_id: L'ID de l'utilisateur pour lequel on veut recommander.
        svd_model (TruncatedSVD): Le modèle SVD entraîné.
        U_latent (np.ndarray): La matrice des utilisateurs dans l'espace latent.
        V_latent (np.ndarray): La matrice des articles dans l'espace latent.
        user_ids (list): La liste des user_ids correspondant à l'index de U.
        article_ids (list): La liste des article_ids correspondant à l'index de V.
        top_n (int): Nombre d'articles à recommander (par défaut 5).

    Returns:
        list: Une liste de top_n click_article_id recommandés.
    """
    # Vérifier si l'utilisateur existe dans les données d'entraînement
    try:
        user_idx = user_ids.index(user_id)
    except ValueError:
        print(f"User ID {user_id} non trouvé dans les données d'entraînement. Impossible de recommander avec SVD.")
        return []

    # Récupérer le vecteur latent de l'utilisateur
    user_vector = U_latent[user_idx, :]

    # Calculer la prédiction de score pour tous les articles pour cet utilisateur
    # Score = U[user_idx, :] @ V.T (produit matriciel)
    # Ou plus simplement, on peut calculer la similarité cosinus entre le vecteur utilisateur
    # et chaque vecteur article dans l'espace latent (moins courant mais possible).
    # La méthode directe : prédiction = U[user] * V^T
    predicted_scores = user_vector @ V_latent.T

    # Créer un dictionnaire pour stocker les scores des articles non lus
    scores = {}

    # Récupérer les articles que l'utilisateur a déjà lus (dans la matrice originale)
    # On suppose que l'on a accès à la matrice originale ici aussi, ou on la reconstruit partiellement
    # Pour simplifier, on suppose que si le score prédit est > 0, on le garde pour trier,
    # mais on devrait idéalement exclure les articles avec une interaction réelle > 0.
    # Pour cela, on aurait besoin de la matrice d'origine ou de l'historique de l'utilisateur.
    # Ici, on trie simplement tous les scores prédits pour cet utilisateur.
    # Une meilleure pratique serait de masquer les articles déjà lus.

    # Trier les articles par score prédit décroissant
    sorted_article_indices = np.argsort(predicted_scores)[::-1]

    # Sélectionner les top_n articles avec les scores prédits les plus élevés
    recommended_article_ids = []
    for idx in sorted_article_indices:
        article_id = article_ids[idx]
        # Optionnel: Exclure les articles déjà lus par l'utilisateur dans les données d'entraînement
        # Cela nécessiterait de récupérer la ligne de l'utilisateur dans la matrice originale
        # if interaction_matrix.loc[user_id, article_id] == 0: # Si non lu
        recommended_article_ids.append(article_id)
        if len(recommended_article_ids) >= top_n:
            break

    # Récupérer les articles déjà lus pour contexte (facultatif)
    # Cela nécessite d'avoir la matrice originale ou de la reconstruire
    # user_interactions = interaction_matrix.loc[user_id]
    # user_read_articles = user_interactions[user_interactions > 0].index.tolist()

    return recommended_article_ids

# --- Exemple d'utilisation ---
if __name__ == "__main__":
    # Charger les données
    clicks_df, interaction_matrix, user_ids, article_ids = load_and_prepare_svd_data(DATA_PATH)

    print(f"Matrice d'interaction chargée. Forme: {interaction_matrix.shape}")
    print(f"Exemples de user_ids: {user_ids[:5]}")
    print(f"Exemples de article_ids: {article_ids[:5]}")

    # Entraîner le modèle SVD
    svd_model, U_latent, V_latent = train_svd_model(interaction_matrix, n_components=50)

    # Sélectionner un utilisateur existant pour tester
    # On prend le premier user_id de la matrice pour s'assurer qu'il existe
    if user_ids:
        test_user_id = user_ids[1]
        print(f"\n--- Test pour l'utilisateur : {test_user_id} ---")
        recommendations = get_svd_recommendations(test_user_id, svd_model, U_latent, V_latent, user_ids, article_ids, top_n=5)
        print(f"Articles recommandés pour l'utilisateur {test_user_id} (SVD): {recommendations}")
    else:
        print("La matrice d'interaction est vide ou ne contient aucun utilisateur.")


Matrice d'interaction chargée. Forme: (707, 323)
Exemples de user_ids: [0, 1, 2, 3, 4]
Exemples de article_ids: [2137, 2662, 4243, 4658, 7947]
Entraînement du modèle SVD avec 50 composantes...
SVD terminé. Forme de U (utilisateurs latent): (707, 50), Forme de V (articles latent): (323, 50)

--- Test pour l'utilisateur : 1 ---
Articles recommandés pour l'utilisateur 1 (SVD): [96663, 235840, 270225, 234995, 270564]
